In [2]:
from src.utils.dataloaders import load_dataloader
from src.models.csnn import ShallowCSNN
from torchsummary import summary

data_params = {
    'data_dir': './data',
    'image_size': 32,
    'batch_size': 32,
    'channels': 1,
    'mean': 0.5,
    'std': 0.5
}

train_dl, val_dl, test_dl = load_dataloader('CIFAR10', data_params, True)

ModuleNotFoundError: No module named 'src.models.csnn'

In [ ]:
model = ShallowCSNN((1, 32, 32), 4, 20, 0.5, None).to('cuda')
for data, target in train_dl:
    data, target = data.to('cuda'), target.to('cuda')
    spk, mem = model(data)
    print(spk.max())
    break

metatensor(0., device='cuda:0', grad_fn=<AliasBackward0>)


: 

In [ ]:
from src.utils.dataloaders import load_dataloader
from tqdm.notebook import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from spikingjelly.activation_based import neuron, functional, surrogate, layer
from torch.utils.tensorboard import SummaryWriter
import os
import time
import argparse
from torch.cuda import amp
import sys
import datetime
from src.models.jelly import SJellyCSNN, VGGLike

data_params = {
    'data_dir': './data',
    'image_size': 32,
    'batch_size': 128,
    'channels': 1,
    'mean': 0.5,
    'std': 0.5
}

device = 'cuda'
train_dl, val_dl, test_dl = load_dataloader('FashionMNIST', data_params, True)
epochs = 10
net = VGGLike((1, 32, 32), 10, 5).to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
scaler = None
start_time = time.time()
np.int = np.int64

for epoch in range(epochs):
    start_time = time.time()
    net.train()
    train_loss = 0
    train_acc = 0
    train_samples = 0

    with tqdm(total=len(train_dl)) as pbar:
        for img, label in train_dl:
            optimizer.zero_grad()
            img = img.to(device)
            label = label.to(device)
            label_onehot = F.one_hot(label, 10).float()

            out_fr = net(img)
            loss = F.mse_loss(out_fr, label_onehot)
            loss.backward()
            optimizer.step()

            train_samples += label.numel()
            train_loss += loss.item() * label.numel()
            train_acc += (out_fr.argmax(1) == label).float().sum().item()

            functional.reset_net(net)
            pbar.update()

    train_time = time.time()
    train_speed = train_samples / (train_time - start_time)
    train_loss /= train_samples
    train_acc /= train_samples
    print(f'Train Loss : {train_loss} | Train Acc : {train_acc}')

  0%|          | 0/375 [00:00<?, ?it/s]

KeyboardInterrupt: 